# Simulated growth in a test tube

# Calculations

In [1]:
# libraries
import numpy as np
from src.evap_standard_curve import *

<div class="alert alert-success">
    Modify your parameters here
</div>

In [2]:
# parameters

# ! neither v_s or v_d should > 10 ul since we have a p10 pipette
tau = 6    # hours. doubling time
v_s = 9    # ul. volume per sample
t_s = 4    # hours. time interval between samples
        # rehydration frequency is the same as sampling frequency
v_d = 9    # dump volume. the frequency of dumping is the same as sampling.
        # this is useful to avoid very small initial volume/growth volume
    
SAMPLE_OFFSET = 0    # use if not starting from A1
T_TOT = 72

# constants

V_MIN = 2    # ul. minimal volume the pipette can handle in one operation
T_MIN = 0.5    # hours. minimal time interval between contiguous steps in a sequence, e.g., two  rehydration steps
RM_OFFSET = 0.5    # ul. essentially the amount of residual liquid on the tip. Subtract this amount from
                    # the "removing" steps

In [3]:
# find out the "balancing" initial volume

# v_removed = v_b * ln(2) * t_s / tau
# where v_b is the balancing volume

v_removed = v_s + v_d    # in time step t_s, how much volume is removed
v_b = v_removed * tau / (np.log(2) * t_s)
v_b = np.round(v_b, decimals=1)    # restricted by precision of the pipette

print("Initial volume should be", v_b, "ul")

Initial volume should be 39.0 ul


In [4]:
# find the "growth" interval
# notice that the strategy is only a function of sampling vol and interval, and discard vol, but not tau
# (tau determines initial vol)

v_removed = v_s + v_d    # which is also the total growth vol

max_n_pip = int(np.floor(t_s / T_MIN))    # max number of pipetting steps, restricted by T_MIN

v_grow = -1    # flag for no strategy
# starting from the max # pipettes
for n_pip in range(max_n_pip + 1, 1, -1):
    
    # i must be a factor of max__pip
    if max_n_pip % n_pip != 0:
        continue
        
    v_cur = np.round(v_removed / n_pip, decimals=1)
    #print("Trying 'growth' volume", v_cur, "ul, pipetting", n_pip, "times per sampling...")
    
    if v_cur >= 3 * V_MIN and v_cur <= 10:
        # the smallest v_grow satisfies min. pipetting precision is what we want
        # v_grow consists of 3 pipetting steps, thus x3
        v_grow = v_cur
        break

if v_grow == -1:
    print("No 'growth' volume found!")
else:
    print("Given that", v_removed, "ul is removed per", t_s, "hours",
          ", the total volume should 'grow' by", v_grow, "ul for", n_pip, "times (interval =",
          t_s/n_pip, "hours) in the same period.")

Given that 18 ul is removed per 4 hours , the total volume should 'grow' by 9.0 ul for 2 times (interval = 2.0 hours) in the same period.


In [5]:
# find the rehydration vol

if t_s == 2:
    v_hydrate = evap_vol_2h(v_s / 3 * 4)
elif t_s == 4:
    v_hydrate = evap_vol_4h(v_s / 3 * 4)
else:
    raise Exception("Rehydration interval can be only either 2 or 4 hours")

print("Rehydration vol each time is", v_hydrate)

Rehydration vol each time is 5.1


# Initialization

In [9]:
# current working dir is /var/lib/jupyter/notebooks
import numpy as np
import opentrons.execute
from opentrons import protocol_api
from src.pipette_viscous import transfer_viscous,aspirate_viscous, dispense_viscous, calibrated_viscous
import src.scheduler as scheduler
from src.evap_standard_curve import *

# start the protocol context
protocol = opentrons.execute.get_protocol_api("2.11")

# home is required
protocol.set_rail_lights(False)
protocol.home()

## Load labware

- The incubator is on slot 11
- The alluminum rack is mounted to the incubator
- The GEB tiprack is on slot 2, 5, 6, 8, 9
- The Bio-rad PCR plate is on slot 3
- The P10 1st gen pipette is on the left

In [10]:
incubator = protocol.load_module("temperature module", 11)

rack = incubator.load_labware("opentrons_24_aluminumblock_fisher_1.5ml_centrifuge")    # the rack is mounted upon the temp. module (don't specify slot)

tip_rack_1 = protocol.load_labware("geb_taller_96_tiprack_10ul", '6')    # will consume 380 tips in total, about 4 boxes
tip_rack_2 = protocol.load_labware("geb_taller_96_tiprack_10ul", '7')
tip_rack_3 = protocol.load_labware("geb_taller_96_tiprack_10ul", '8')
tip_rack_4 = protocol.load_labware("geb_taller_96_tiprack_10ul", '9')
#tip_rack_5 = protocol.load_labware("geb_taller_96_tiprack_10ul", '4')    # custom labware. see definition at labware/

plate = protocol.load_labware("biorad_96_wellplate_200ul_pcr", '5')

pipette = protocol.load_instrument("p10_single", "left", 
                                   tip_racks = [tip_rack_1, tip_rack_2, tip_rack_3, tip_rack_4])

# Define liquid/tubes

In [11]:
RXN_TUBE = rack.wells_by_name()["A1"]
DYE_TUBE = rack.wells_by_name()["A3"]
WAT_TUBE = rack.wells_by_name()["A5"]
DIS_TUBE = rack.wells_by_name()["C1"]    # discard
A_TUBE = rack.wells_by_name()["C3"]    # 3x KaiA
B_TUBE = rack.wells_by_name()["C4"]    # 3x KaiB
U_TUBE = rack.wells_by_name()["C5"]    # 3x unphosphorylated KaiC

# Define operations

In [12]:
def sample_and_discard(idx):
    "take a sample from the tube to a 96-well plate"

    # current well
    cur_well = plate.wells()[idx + SAMPLE_OFFSET]

    # pipette the dye
    transfer_viscous(pipette, protocol, calibrated_viscous(v_s / 3), DYE_TUBE, cur_well)

    # pipette the sample
    aspirate_viscous(pipette, protocol, calibrated_viscous(v_s - RM_OFFSET), RXN_TUBE, asp_height=2)
    dispense_viscous(pipette, protocol, calibrated_viscous(v_s - RM_OFFSET), cur_well, if_mix=True)
    
    # discard from reaction
    # no need to discard at time 0
    if idx > 0:
        aspirate_viscous(pipette, protocol, calibrated_viscous(v_d - RM_OFFSET), RXN_TUBE, asp_height = 2)
        pipette.dispense(calibrated_viscous(v_d - RM_OFFSET), DIS_TUBE)
        pipette.drop_tip()

# for logging
str_sample_and_discard = "Sample and mix with loading buffer, and discard from the reaction"

def rehydrate(idx):
    "to prevent drying down"

    # determine rehydration vol
    if t_s == 2:
        v_hydrate = evap_vol_2h(v_s / 3 * 4)
    elif t_s == 4:
        v_hydrate = evap_vol_4h(v_s / 3 * 4)
    else:
        raise Exception("Rehydration interval can be only either 2 or 4 hours")
    
    for i in range(idx):
        cur_well = plate.wells()[i + SAMPLE_OFFSET]
        
        pipette.pick_up_tip()
        pipette.aspirate(calibrated_viscous(v_hydrate), WAT_TUBE)    # just water, no oil
        dispense_viscous(pipette, protocol, calibrated_viscous(v_hydrate), cur_well, if_mix=True)
        
str_rehydrate = "Rehydrate to prevent drying down"
        
def grow(v_g):
    "dilute the reaction with U-KaiC, KaiA, and KaiB by volume v_g"
    
    v_g_per_tube = np.round(v_g / 3, 1)
    
    for from_tube in [U_TUBE, A_TUBE, B_TUBE]:
        # it's also okay to mix KaiA and KaiB in one tube, then all conc are 2x
        # do not dispense at the bottom. Defaults are 1mm from the bottom
        aspirate_viscous(pipette, protocol, calibrated_viscous(v_g_per_tube), from_tube, asp_height=2)
        dispense_viscous(pipette, protocol, calibrated_viscous(v_g_per_tube), RXN_TUBE, if_mix=True)
        # here is a decision: now I do a blow out. A blow out can push more liquid out of the tip but
        # introduces a bubble. Probably worth it?
    
str_grow = "Simulate growth"

# Define instructions

## This is a test for each operation

In [7]:
# scheduler.drop()
# scheduler.cat(time_vec=[0, 0, 0, 0],
#              func_vec=[sample_and_discard, sample_and_discard, rehydrate, grow],
#              param_vec=[(0,), (1,), (2,), (4.5,)],
#              str_vec=[str_sample_and_discard, str_sample_and_discard, str_rehydrate, str_grow],
#              n_tip_vec=[3, 3, 2, 3])
# scheduler.report(unit="hours")

A total of 9 tips is required

At 0.00 hours, Sample and mix with loading buffer, and discard from the reaction, with params (0,)
At 0.00 hours, Sample and mix with loading buffer, and discard from the reaction, with params (1,)
At 0.00 hours, Rehydrate to prevent drying down, with params (2,)
At 0.00 hours, Simulate growth, with params (4.5,)


## The real instructions

In [13]:
incubator.set_temperature(30)

In [14]:
scheduler.drop()

# t_s, v_s, and v_d are previously defined
# v_grow is calculated in one of the above cells

# growth
t_grow = t_s / n_pip
n_steps_grow = int(T_TOT / t_grow)

scheduler.cat(time_vec=np.arange(1, n_steps_grow + 1) * t_grow * 60,
             func_vec=[grow] * n_steps_grow,
             param_vec=[(v_grow,)] * n_steps_grow,    # dilution volumes are all the same
             str_vec=[str_grow] * n_steps_grow,
             n_tip_vec=[3] * n_steps_grow)

# sampling
n_steps_sample = int(T_TOT / t_s) + 1    # plus time 0

scheduler.cat(time_vec=np.arange(n_steps_sample) * t_s * 60,
             func_vec=[sample_and_discard] * n_steps_sample,
             param_vec=[ (i,) for i in range(n_steps_sample) ],
             str_vec=[str_sample_and_discard] * n_steps_sample,
             n_tip_vec=[2] + [3] * (n_steps_sample - 1))

# rehydration
scheduler.cat(time_vec=np.arange(1, n_steps_sample) * t_s * 60,
             func_vec=[rehydrate] * (n_steps_sample - 1),
             param_vec=[ (i,) for i in range(1, n_steps_sample) ],
             str_vec=[str_rehydrate] * (n_steps_sample - 1),
             n_tip_vec=[ i for i in range(1, n_steps_sample) ])

scheduler.report(unit="hours")

A total of 335 tips is required

At 2.00 hours, Simulate growth, with params (9.0,)
At 4.00 hours, Simulate growth, with params (9.0,)
At 6.00 hours, Simulate growth, with params (9.0,)
At 8.00 hours, Simulate growth, with params (9.0,)
At 10.00 hours, Simulate growth, with params (9.0,)
At 12.00 hours, Simulate growth, with params (9.0,)
At 14.00 hours, Simulate growth, with params (9.0,)
At 16.00 hours, Simulate growth, with params (9.0,)
At 18.00 hours, Simulate growth, with params (9.0,)
At 20.00 hours, Simulate growth, with params (9.0,)
At 22.00 hours, Simulate growth, with params (9.0,)
At 24.00 hours, Simulate growth, with params (9.0,)
At 26.00 hours, Simulate growth, with params (9.0,)
At 28.00 hours, Simulate growth, with params (9.0,)
At 30.00 hours, Simulate growth, with params (9.0,)
At 32.00 hours, Simulate growth, with params (9.0,)
At 34.00 hours, Simulate growth, with params (9.0,)
At 36.00 hours, Simulate growth, with params (9.0,)
At 38.00 hours, Simulate growth, wi

<div class="alert alert-success">
    Prepare the following liquid
</div>

In [15]:
V_SAFE = 50

v_rxn = v_b + v_s    # time 0 sample
v_dye = v_s / 3 * n_steps_sample + V_SAFE
v_wat = v_hydrate * sum(range(1, n_steps_sample)) + V_SAFE
# actually water tube is 1 ml, without oil, and refilled every day
# water costs nothing...
v_stock = v_grow / 3 * n_steps_grow + V_SAFE

print(f"Reaction tube at A1, {v_rxn:.1f} ul")
print(f"Dye tube at A3, {v_dye:.1f} ul")
print(f"Water tube at A5, {v_wat:.1f} ul")
print(f"Stock protein tube at C3, C4, and C5, {v_stock:.1f} ul")

Reaction tube at A1, 48.0 ul
Dye tube at A3, 107.0 ul
Water tube at A5, 922.1 ul
Stock protein tube at C3, C4, and C5, 158.0 ul


<div class="alert alert-warning">
    <h2>STOP!</h2>
    Before you proceed, please check:<br />
    Have you placed the required labware?<br />
    Have you put the right volume of liquid in required position?
</div>

In [ ]:
log_fn = "log/20221004_simulated_growth.log"
scheduler.run(protocol, log_fn)

## A quicker test

In [11]:
v_s = 9
v_d = 9

def sample_test(idx):
    "take a sample from the tube to a 96-well plate"

    # current well
    cur_well = plate.wells()[idx + SAMPLE_OFFSET]

    # pipette the dye
    #transfer_viscous(pipette, protocol, v_s / 3, DYE_TUBE, cur_well)

    # pipette the sample
    aspirate_viscous(pipette, protocol, calibrated_viscous(v_s - RM_OFFSET), RXN_TUBE, asp_height=2)
    dispense_viscous(pipette, protocol, calibrated_viscous(v_s - RM_OFFSET), cur_well, if_mix=True)
    
    # discard from reaction
    # no need to discard at time 0
    if idx > 0:
        aspirate_viscous(pipette, protocol, calibrated_viscous(v_d - RM_OFFSET), RXN_TUBE, asp_height=2)
        pipette.dispense(calibrated_viscous(v_d - RM_OFFSET), DIS_TUBE)
        pipette.drop_tip()

# for logging
str_sample_and_discard = "Sample and mix with loading buffer, and discard from the reaction"


def grow_test(v_g):
    "dilute the reaction with U-KaiC, KaiA, and KaiB by volume v_g"
    
    v_g_per_tube = np.round(v_g / 3, 1)
    
    for from_tube in [U_TUBE, A_TUBE, B_TUBE]:
        # it's also okay to mix KaiA and KaiB in one tube, then all conc are 2x
        # do not dispense at the bottom. Defaults are 1mm from the bottom
        aspirate_viscous(pipette, protocol, calibrated_viscous(v_g_per_tube), from_tube, asp_height=2)
        dispense_viscous(pipette, protocol, calibrated_viscous(v_g_per_tube), RXN_TUBE, if_mix=True)
    
str_grow = "Simulate growth"

In [12]:
n_steps_sample = 19
n_steps_grow = 18 * 2
t_sample = 20    # minutes
t_grow = 10
v_grow = 9    # ul

v_g = 9

scheduler.drop()
scheduler.cat(time_vec=(np.arange(n_steps_grow) + 1) * t_grow,
             func_vec=[grow_test] * n_steps_grow,
             param_vec=[(v_g,)] * n_steps_grow,
             str_vec=[str_grow] * n_steps_grow,
             n_tip_vec=[3] * n_steps_grow)
scheduler.cat(time_vec=np.arange(n_steps_sample) * t_sample,
             func_vec=[sample_test] * n_steps_sample,
             str_vec=[str_sample_and_discard] * n_steps_sample,
             n_tip_vec=[1] + [2] * (n_steps_sample - 1),
             param_vec=[(i,) for i in range(n_steps_sample)])
scheduler.report(unit="minutes")

A total of 145 tips is required

At 10 minutes, Simulate growth, with params (9,)
At 20 minutes, Simulate growth, with params (9,)
At 30 minutes, Simulate growth, with params (9,)
At 40 minutes, Simulate growth, with params (9,)
At 50 minutes, Simulate growth, with params (9,)
At 60 minutes, Simulate growth, with params (9,)
At 70 minutes, Simulate growth, with params (9,)
At 80 minutes, Simulate growth, with params (9,)
At 90 minutes, Simulate growth, with params (9,)
At 100 minutes, Simulate growth, with params (9,)
At 110 minutes, Simulate growth, with params (9,)
At 120 minutes, Simulate growth, with params (9,)
At 130 minutes, Simulate growth, with params (9,)
At 140 minutes, Simulate growth, with params (9,)
At 150 minutes, Simulate growth, with params (9,)
At 160 minutes, Simulate growth, with params (9,)
At 170 minutes, Simulate growth, with params (9,)
At 180 minutes, Simulate growth, with params (9,)
At 190 minutes, Simulate growth, with params (9,)
At 200 minutes, Simulate g

In [18]:
protocol.home()

In [ ]:
log_fn = "log/20220909_test_simulated_growth.log"
scheduler.run(protocol, log_fn)